In [2]:
import cv2
import face_recognition
import numpy as np
from keras.preprocessing import image
from keras.models import model_from_json

In [5]:
face_exp_model=model_from_json(open('C:/Users/ASUS/Desktop/face recognition/datasets/facial_expression_model_structure.json',"r").read())
face_exp_model.load_weights('C:/Users/ASUS/Desktop/face recognition/datasets/facial_expression_model_weights.h5')
emotion_label=('angry','disgust','fear','happy','sad','surprise','neutral')
webcam_video=cv2.VideoCapture(0)
number_of_faces=[]
while True:
   #get the current frame of video stream as an image
    ret,current_frame=webcam_video.read()
    #resize the current frame to 1/4 to process faster
    resize_frame=cv2.resize(current_frame,(0,0),fx=0.25,fy=0.25)
    number_of_faces=face_recognition.face_locations(resize_frame,number_of_times_to_upsample=2,model='hog')
    for index,current_face_location in enumerate(number_of_faces):
        top_pos, right_pos, bottom_pos, left_pos = current_face_location
        top_pos = top_pos*4
        right_pos = right_pos*4
        bottom_pos = bottom_pos*4
        left_pos = left_pos*4
        print("Found face {} at top:{} , right:{} , bottom:{} , left:{}".format(index+1, top_pos, right_pos,
                                                                        bottom_pos, left_pos))
        cv2.rectangle(current_frame,(left_pos,top_pos),(right_pos,bottom_pos),(0,255,255),2)
        current_face_image = current_frame[top_pos:bottom_pos, left_pos:right_pos]
        #preprocess the input image, convert into grayscale image
        current_face_image = cv2.cvtColor(current_face_image,cv2.COLOR_BGR2GRAY)
        #resize to 48*48 px size
        current_face_image = cv2.resize(current_face_image, (48,48))
        #convert the PIL image into numpy array
        input_image = image.img_to_array(current_face_image)
        #convert into 1d array
        input_image = np.expand_dims(input_image, axis=0)
        input_image = input_image/255
        
        expression = face_exp_model.predict(input_image)
        max_index = np.argmax(expression[0])
        print(max_index)
        emotion_label1 = emotion_label[max_index]
        
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(current_frame, emotion_label1, (left_pos,bottom_pos), font, 1, (255,255,255), 1)
    cv2.imshow("Webcam Video", current_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam_video.release()
cv2.destroyAllWindows()
        
        
    
